In [1]:
import tensorflow as tf
from tensorflow.keras import layers,models
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt

In [ ]:
# transfer learning
base_model = tf.keras.applications.VGG16(input_shape=(224, 224, 3), include_top=False)  #224 pixels * 224 pixels with 3 color channels (RGB)
# include_top=False means we don't include the final classification layers of VGG16 so reduce computation
for layer in base_model.layers:
    layer.trainable = False

model = Sequential([
    base_model,
    Flatten(),  # Flatten the output of the base model to one dimension 
    Dense(128, activation='relu'),  # optional hidden layer
    Dense(5, activation='softmax')  # main output layer for 5 classes 
])
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
                metrics=['accuracy'])
model.summary()

In [ ]:
(train,test),ds_info=tfds.load('food101',split=['train','validation'],as_supervised=True,with_info=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

In [3]:
ds_info

tfds.core.DatasetInfo(
    name='food101',
    full_name='food101/2.0.0',
    description="""
    This dataset consists of 101 food categories, with 101'000 images. For each class, 250 manually reviewed test images are provided as well as 750 training images. On purpose, the training images were not cleaned, and thus still contain some amount of noise. This comes mostly in the form of intense colors and sometimes wrong labels. All images were rescaled to have a maximum side length of 512 pixels.
    """,
    homepage='https://data.vision.ee.ethz.ch/cvl/datasets_extra/food-101/',
    data_dir='/root/tensorflow_datasets/food101/2.0.0',
    file_format=tfrecord,
    download_size=4.65 GiB,
    dataset_size=4.77 GiB,
    features=FeaturesDict({
        'image': Image(shape=(None, None, 3), dtype=uint8),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=101),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    nondeterministic_order=False,
    split

In [3]:
NUM_CLASSES=ds_info.features['label'].num_classes
NUM_CLASSES

101

In [5]:
train

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, None, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [4]:
IMG_SIZE=(224,224)

In [5]:
def preprocess_img(image,label):
    image=tf.image.resize(image,IMG_SIZE)
    image=tf.cast(image,tf.float32)/255.0
    return image,label

In [6]:
train=train.map(preprocess_img)
test=test.map(preprocess_img)

In [7]:
data_augmentation=tf.keras.Sequential(
    [
        
        layers.RandomFlip('horizontal'),
        layers.RandomRotation(0.15),
        layers.RandomZoom(0.2),
        layers.RandomContrast(0.2)
    ]
)

In [8]:
def conv_block(x,filters):
    x=layers.Conv2D(filters,(3,3))(x)
    x=layers.BatchNormalization()(x)
    x=layers.ReLU()(x)
    x=layers.MaxPooling2D()(x)
    return x


In [9]:
inputs=layers.Input(shape=(224,224,3))
x=data_augmentation(inputs)

x=conv_block(x,32)
x=conv_block(x,64)
x=conv_block(x,128)
x=conv_block(x,256)


x=layers.Flatten()(x)

x=layers.Dense(256,activation='relu')(x)
x=layers.Dropout(0.3)(x)
outputs=layers.Dense(NUM_CLASSES,activation='softmax')(x)


model=models.Model(inputs,outputs)
model.summary()



Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential (Sequential)         │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 222, 222, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu (ReLU)                    │ (None, 222, 222, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 109, 109, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_1 (ReLU)                  │ (None, 109, 109, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 52, 52, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_2 (ReLU)                  │ (None, 52, 52, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 24, 24, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 24, 24, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_3 (ReLU)                  │ (None, 24, 24, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 12, 12, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 36864)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │     9,437,440 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 101)            │        25,957 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,853,733 (37.59 MB)

 Trainable params: 9,852,773 (37.59 MB)

 Non-trainable params: 960 (3.75 KB)

In [12]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [13]:
history=model.fit(train,epochs=5,validation_data=test)

Epoch 1/5
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 384s 157ms/step - accuracy: 0.0090 - loss: 5.0740 - val_accuracy: 0.0099 - val_loss: 4.6152
Epoch 2/5
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 371s 156ms/step - accuracy: 0.0085 - loss: 4.6160 - val_accuracy: 0.0099 - val_loss: 4.6152
Epoch 3/5
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 372s 156ms/step - accuracy: 0.0082 - loss: 4.6159 - val_accuracy: 0.0099 - val_loss: 4.6152
Epoch 4/5
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 372s 156ms/step - accuracy: 0.0083 - loss: 4.6160 - val_accuracy: 0.0099 - val_loss: 4.6152
Epoch 5/5
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 373s 156ms/step - accuracy: 0.0079 - loss: 4.6159 - val_accuracy: 0.0099 - val_loss: 4.6152


In [10]:
train=(train.shuffle(1000).batch(32).prefetch(tf.data.AUTOTUNE))

In [11]:
test=(test.shuffle(1000).batch(32).prefetch(tf.data.AUTOTUNE))